In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import train_test_split

# from swintransformer import SwinTransformer
import tensorflow as tf
from tensorflow.keras import callbacks
import tensorflow.keras.layers as tfl
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom, RandomTranslation
import tensorflow.keras.backend as K

import cv2

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2021-10-30 10:35:15.336526: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import sys
sys.path.append('../input/swintransformertf')
from swintransformer import SwinTransformer

In [3]:
train_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')

In [4]:
BATCH_SIZE = 64
IMG_SIZE = (384, 384)
directory = '/kaggle/input/petfinder-pawpularity-score/train/'
seed = 1022
fold = 3
epochs = 10
learning_rate = 0.001
splitted_directory = '/kaggle/input/data-split-pawpularity/'

In [7]:
def preprocess_dataset(train_df, val_df, img_size=IMG_SIZE, batch_size= BATCH_SIZE, seed=seed, buffer_size = tf.data.experimental.AUTOTUNE):
    X_train = tf.constant(train_df['img_path'])
    y_train = tf.constant(train_df['Pawpularity'])
    X_val = tf.constant(val_df['img_path'])
    y_val = tf.constant(val_df['Pawpularity'])
    
    print('Training Set = ', X_train.shape)
    print('Validation Set = ', X_val.shape)
    
    def preprocess(image_url, label):
        image_string = tf.io.read_file(image_url)
        image = tf.image.decode_jpeg(image_string, channels=3)
        image = tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(image, tf.float32), mode="torch")
        image = tf.image.central_crop(image, 1.0)
        image = tf.image.resize(image, img_size)
        return image, label
    
    training_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).map(preprocess).batch(batch_size).prefetch(buffer_size=buffer_size)
    val_data = tf.data.Dataset.from_tensor_slices((X_val, y_val)).map(preprocess).batch(batch_size).prefetch(buffer_size=buffer_size)
    
    return training_data, val_data

In [8]:
def data_augmenter():
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip('horizontal'))
    data_augmentation.add(RandomRotation(0.2))
    
    return data_augmentation

In [9]:
def build_model(image_size = IMG_SIZE, data_augmentation = data_augmenter()):

    input_shape = IMG_SIZE + (3,)
    base_model = SwinTransformer('swin_large_384', include_top=False, pretrained=True)
    base_model.trainable = False

    inputs = tf.keras.Input(shape=input_shape) 
    x = data_augmentation(inputs)
    x = base_model(x, training=False)
    x = tfl.Dropout(0.5)(x)

    x = tfl.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = tfl.BatchNormalization()(x)
    x = tfl.LeakyReLU()(x)
    x = tfl.Dropout(0.5)(x)
    
    x = tfl.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = tfl.BatchNormalization()(x)
    x = tfl.LeakyReLU()(x)
    x = tfl.Dropout(0.5)(x)
    
    outputs = tfl.Dense(1, activation = 'sigmoid')(x)
    
    model = tf.keras.Model(inputs, outputs)
    return model

2021-10-30 10:35:20.389147: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-30 10:35:20.392993: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-10-30 10:35:20.434558: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-30 10:35:20.435703: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-10-30 10:35:20.435759: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-10-30 10:35:20.466385: I tensorflow/stream_executor/platform/def

In [10]:
def RMSE(y_true, y_pred, denormalize=True):
    if denormalize:
        # denormalizing
        y_true = y_true*100.0
        y_pred = y_pred*100.0
    # rmse
    loss = tf.math.sqrt(tf.math.reduce_mean(tf.math.square(tf.subtract(y_true, y_pred))))
    return loss
RMSE.__name__='rmse'

rmse = RMSE

In [12]:
history_folds = []

In [13]:
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [14]:
rlr = callbacks.ReduceLROnPlateau(monitor="val_rmse",factor=0.3, patience=3)
es = tf.keras.callbacks.EarlyStopping(monitor='val_rmse', patience=3, restore_best_weights=True)

In [15]:
for i in range(fold):
    train_df = pd.read_csv(splitted_directory + f'skf_split_train_5cv_fold{i}.csv')
    valid_df = pd.read_csv(splitted_directory + f'skf_split_val_5cv_fold{i}.csv')
    
    train, val = preprocess_dataset(train_df, valid_df)
    
    K.clear_session()
    model = build_model(data_augmentation= data_augmenter())
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
          loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.01),
          metrics=[rmse])
    model.summary()
    history = model.fit(train, validation_data=val, epochs=epochs, callbacks=[rlr, es])
    
    model.layers[2].trainable = True
    for j in model.layers[2].layers[2].layers[:3]:
        j.trainable = False
    model.layers[2].layers[2].layers[3].trainable = True
    print(model.layers[2].layers[2].layers[3].trainable)
    model.layers[2].layers[0].trainable = False
    model.layers[2].layers[1].trainable = False
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate/100),
          loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.01),
          metrics=[rmse])
    model.summary()
    history_finetuned = model.fit(train, validation_data=val, initial_epoch=history.epoch[-1], epochs=epochs*2, callbacks=[rlr, es])
    
    model.save(f'model_fold{i}')
    
    history_folds.append(history_finetuned.history)

Training Set =  (7929,)
Validation Set =  (1983,)
723476480/723475820 [==============================] - 10s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 384, 384, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 384, 384, 3)       0         
_________________________________________________________________
swin_large_384 (SwinTransfor (None, 1536)              198101556 
_________________________________________________________________
dropout_73 (Dropout)         (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               786944    
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________

2021-10-30 10:36:02.129144: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-30 10:36:02.132702: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz
2021-10-30 10:36:36.042881: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-10-30 10:36:37.058875: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-10-30 10:36:38.388909: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


124/124 [==============================] - 665s 5s/step - loss: 1.7151 - rmse: 32.7903 - val_loss: 1.3839 - val_rmse: 19.4891
Epoch 2/10
124/124 [==============================] - 618s 5s/step - loss: 1.3892 - rmse: 24.8163 - val_loss: 1.1732 - val_rmse: 18.3886
Epoch 3/10
124/124 [==============================] - 617s 5s/step - loss: 1.1630 - rmse: 22.3042 - val_loss: 1.0007 - val_rmse: 18.1999
Epoch 4/10
124/124 [==============================] - 617s 5s/step - loss: 0.9841 - rmse: 20.4964 - val_loss: 0.8770 - val_rmse: 18.2366
Epoch 5/10
124/124 [==============================] - 617s 5s/step - loss: 0.8642 - rmse: 19.6959 - val_loss: 0.7950 - val_rmse: 18.1725
Epoch 6/10
124/124 [==============================] - 617s 5s/step - loss: 0.7889 - rmse: 19.4013 - val_loss: 0.7448 - val_rmse: 18.1432
Epoch 7/10
124/124 [==============================] - 617s 5s/step - loss: 0.7401 - rmse: 18.8606 - val_loss: 0.7159 - val_rmse: 18.1689
Epoch 8/10
124/124 [==============================] 

2021-10-30 14:22:18.539096: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Training Set =  (7929,)
Validation Set =  (1983,)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 384, 384, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 384, 384, 3)       0         
_________________________________________________________________
swin_large_384 (SwinTransfor (None, 1536)              198101556 
_________________________________________________________________
dropout_73 (Dropout)         (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               786944    
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
leaky_re_lu

124/124 [==============================] - 661s 5s/step - loss: 1.6732 - rmse: 30.7656 - val_loss: 1.3623 - val_rmse: 18.3611
Epoch 2/10
124/124 [==============================] - 618s 5s/step - loss: 1.3694 - rmse: 24.2048 - val_loss: 1.1498 - val_rmse: 18.1587
Epoch 3/10
124/124 [==============================] - 618s 5s/step - loss: 1.1333 - rmse: 21.5852 - val_loss: 0.9778 - val_rmse: 18.3188
Epoch 4/10
124/124 [==============================] - 619s 5s/step - loss: 0.9594 - rmse: 20.1816 - val_loss: 0.8557 - val_rmse: 18.2170
Epoch 5/10
124/124 [==============================] - 620s 5s/step - loss: 0.8432 - rmse: 19.3961 - val_loss: 0.7794 - val_rmse: 18.1810
True
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 384, 384, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 384, 3

In [ ]:
plt.figure(figsize=(25, 25))

for idx, hist in enumerate(history_folds):
    rmse = [0.] + hist['rmse']
    val_rmse = [0.] + hist['val_rmse']
    
    loss = hist['loss']
    val_loss = hist['val_loss']
    
    plt.subplot(5, 2, idx*2+1)
    plt.plot(rmse, label='Training RMSE')
    plt.plot(val_rmse, label='Validation RMSE')
    plt.legend(loc='lower right')
    plt.ylabel('RMSE')
    # plt.ylim([min(plt.ylim()),1])
    plt.title(f'Fold-{idx} Training and Validation RMSE')
    
    plt.subplot(5, 2, idx*2+2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    # plt.ylim([0,1.0])
    plt.title(f'Fold-{idx} Training and Validation Loss')
    plt.xlabel('epoch')

In [ ]:
print('### CV scores by fold ###')
for i in range(fold):
    hist = history_folds[i]
    print(f'fold {i+1}: {hist["val_rmse"][-1]}')
print('CV mean score: {0:.4f}, std: {1:.4f}.'\
      .format(np.mean([i['val_rmse'][-1] for i in history_folds]), np.std([i['val_rmse'][-1] for i in history_folds])))

In [ ]:
def preprocess_test(image_url):
    image_string = tf.io.read_file(image_url)
    image = tf.image.decode_jpeg(image_string, channels=3)
#     image = tf.cast(image, tf.float32)
    image = tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(image, tf.float32), mode="torch")
    image = tf.image.central_crop(image, 1.0)
    image = tf.image.resize(image, IMG_SIZE)
    return image

In [ ]:
for _, _, id in os.walk('/kaggle/input/petfinder-pawpularity-score/test/'):
    ids = id

fileNames = ['/kaggle/input/petfinder-pawpularity-score/test/'+i for i in ids]
test = tf.data.Dataset.from_tensor_slices(fileNames).map(preprocess_test).batch(BATCH_SIZE)

In [ ]:
dependencies = {
    'rmse': rmse
}

In [ ]:
predictions = []
for i in range(fold):
    K.clear_session()
    model = tf.keras.models.load_model(f'./model_fold{i}', custom_objects=dependencies)
    pred = (model.predict(test) * 100.00).reshape(-1)
    predictions.append(pred)
    print(f'fold {i}', pred)

In [ ]:
average_predictions = list(pd.DataFrame(predictions).T.mean(axis = 1))
print(average_predictions)

In [ ]:
ids = [i.split('.')[0] for i in ids]
pred_df = pd.DataFrame({'Id':ids,
                        'Pawpularity':average_predictions})
sub_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
del sub_df['Pawpularity']
sub_df = sub_df.merge(pred_df, on='Id', how='left')
display(sub_df)
sub_df.to_csv('submission.csv',index=False)
sub_df.head(2)